# Constants:

In [1]:
SEQ_LEN = 90
CANDLES_SHIFT = 2 # how many candles to shift between sequences
NAME = "5timeDecay_m30_ov40th015p_shift2_80"
VALIDATION_PCT = 0.2

# Functions:

compose sequences df

In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd

# composes dfs of individual sequences from different timescales
# dfs should be ordered fine to coarse with the finest one having the labels
# seqLen should be divisible by the number of time-resolutions len(dfs)
def composeTimeDecaySequences(dfs, timescales):
    numTimeScales = len(dfs)
    seqPartLen = int(SEQ_LEN/numTimeScales)

    sequences = []
    finestEndIndex = len(dfs[0])-1
    numSequences = int((len(dfs[0]) - int(seqPartLen*(sum(timescales))/timescales[0]))/CANDLES_SHIFT)
    for _ in tqdm(range(numSequences)):

        lastTimestamp = dfs[0]["ts_closeTime"].iloc[finestEndIndex]+1
        seqDf = pd.DataFrame()
        for i in range(numTimeScales):

            # get prev index. Last with ts_closeTime < lastTimestamp
            endIndex = dfs[i][dfs[i]["ts_closeTime"] < lastTimestamp].index[-1]
            seqDf = pd.concat([dfs[i][endIndex-seqPartLen:endIndex], seqDf])
            lastTimestamp = seqDf["ts_closeTime"].iloc[0]
        

        label = seqDf['target'].iloc[-1]
        seqDf = seqDf[['close', 'weightedAverage', 'hl_percent', 'quantity_baseUnits', 'timescale']]
        seq = seqDf.values.tolist()
        if(len(seq) == SEQ_LEN):
            sequences.append([seq, label])

        finestEndIndex -= CANDLES_SHIFT

    return sequences[::-1]
    



balance

In [3]:
def balance(seqs):
    
    buys = []
    sells = []
    holds = []
    for seq in seqs:
        if seq[1] == 0:
            sells.append(seq)
        elif seq[1] == 1:
            buys.append(seq)
        else:
          holds.append(seq)

    print("before balancing:")
    print("buys:", len(buys), ", sells:", len(sells), ", holds:", len(holds))

    smallest = min(len(buys), len(sells), len(holds))
    buys = buys[:smallest]
    sells = sells[:smallest]
    holds = holds[:smallest]

    seqsBalanced = buys+sells+holds
    return seqsBalanced

preprocessing

In [4]:
from sklearn.preprocessing import StandardScaler


# Function to preprocess data
def preprocess3_train(df):
    # before sequencing
    #
    # log and pctchange transform price columns ('low', 'high', 'open', 'close')
    # scale every colum (center mean and unit variance)

    scaler_dict = {}
    for col in df.columns:
        
        if col != 'target' and col != 'ts_closeTime':
            if col != 'quantity_baseUnits' and col != 'hl_percent' and col != 'timescale':
                df[col] = np.log(df[col])
                df[col] = df[col].pct_change()
                df.dropna(inplace=True)
            scaler = StandardScaler()
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
            scaler_dict[col] = scaler
    df.index = np.arange(0, len(df))
    return df, scaler_dict

# Function to apply saved preprocessing to new data
def apply_preprocess3_val(df, scaler_dict):
    # before sequencing
    #
    # pct.change transform price columns ('low', 'high', 'open', 'close')
    # scale every colum (center mean and unit variance)
    
    for col in df.columns:
        if col != 'target' and col != 'ts_closeTime':
            if col != 'quantity_baseUnits' and col != 'hl_percent' and col != 'timescale':
                df[col] = np.log(df[col])
                df[col] = df[col].pct_change()
                df.dropna(inplace=True)
            scaler = scaler_dict[col]
            df[col] = scaler.transform(df[col].values.reshape(-1, 1))
    df.index = np.arange(0, len(df))
    return df

X y split

In [5]:
def xySplit(seqWithTarget):

    X = []
    y = []
    for seq, target in seqWithTarget:
        X.append(seq)
        y.append(target)

    return np.array(X),np.array(y)

# DF manipulation, build training sets:

In [6]:
import pandas as pd
df_30m_classified = pd.read_csv("../historicalData/aligned/HistoricalDataLabeled_BTC_USDT_01072016_01072023_MINUTE_30_ov40_th015p.csv")
df_2h = pd.read_csv("../historicalData/aligned/HistoricalData_BTC_USDT_01072016_01072023_HOUR_2.csv")
df_24h = pd.read_csv("../historicalData/aligned/HistoricalData_BTC_USDT_01072016_01072023_DAY_1.csv")

df_30m_classified = df_30m_classified[['ts_closeTime', 'close', 'weightedAverage', 'hl_percent', 'quantity_baseUnits', 'target']]
df_2h = df_2h[['ts_closeTime', 'close', 'weightedAverage', 'hl_percent', 'quantity_baseUnits']]
df_24h = df_24h[['ts_closeTime', 'close', 'weightedAverage', 'hl_percent', 'quantity_baseUnits']]

df_30m_classified["timescale"] = 0.5
df_2h["timescale"] = 2
df_24h["timescale"] = 24

print(df_30m_classified)

        ts_closeTime     close  weightedAverage  hl_percent  \
0         1467504000    705.00           704.63    0.005262   
1         1467505800    705.46           704.65    0.008062   
2         1467507600    702.09           697.78    0.012057   
3         1467509400    700.07           700.08    0.001555   
4         1467511200    697.95           701.96    0.009872   
...              ...       ...              ...         ...   
122601    1688185800  30348.04         30379.52    0.003004   
122602    1688187600  30388.37         30364.50    0.002573   
122603    1688189400  30406.47         30403.91    0.001523   
122604    1688191200  30426.45         30411.08    0.000850   
122605    1688193000  30424.86         30426.49    0.001111   

        quantity_baseUnits  target  timescale  
0                 2.934065       0        0.5  
1                 6.343613       0        0.5  
2                47.244461       0        0.5  
3                 0.548509       0        0.5  
4  

In [7]:
def prep(df):
    # Split data into train and validation sets
    train_size = int((1-VALIDATION_PCT) * len(df))
    train_df = df.iloc[:train_size].copy()
    val_df = df.iloc[train_size:].copy()

    # Preprocess the training data and save the scaling parameters
    train_df, scaler_dict = preprocess3_train(train_df)
    # Apply saved preprocessing to validation data
    val_df = apply_preprocess3_val(val_df, scaler_dict)

    return train_df, val_df


In [8]:
df_30m_classified_train, df_30m_classified_val = prep(df_30m_classified)
df_2h_train, df_2h_val = prep(df_2h)
df_24h_train, df_24h_val = prep(df_24h)


build time decay sequences

In [9]:
seqsTraining = composeTimeDecaySequences([df_30m_classified_train, df_2h_train, df_24h_train], [1800, 7200, 86400])
seqsValidation = composeTimeDecaySequences([df_30m_classified_val, df_2h_val, df_24h_val], [1800, 7200, 86400])

100%|██████████| 11465/11465 [00:35<00:00, 322.85it/s]


balance buys/sells/holds

In [10]:
# balance buys, sells, and holds
import random

random.shuffle(seqsTraining)
random.shuffle(seqsValidation)
seqsTrainingBalanced = balance(seqsTraining)
seqsValidationBalanced = balance(seqsValidation)
random.shuffle(seqsTrainingBalanced)
random.shuffle(seqsValidationBalanced)

before balancing:
buys: 16211 , sells: 14831 , holds: 17155
before balancing:
buys: 3449 , sells: 3489 , holds: 4464


split sequence from label

In [11]:
train_x, train_y = xySplit(seqsTrainingBalanced)
validation_x, validation_y = xySplit(seqsValidationBalanced)

In [12]:
print(type(train_x))
print(train_x.shape)
print(type(validation_x))
print(validation_x.shape)

<class 'numpy.ndarray'>
(44493, 90, 5)
<class 'numpy.ndarray'>
(10347, 90, 5)


In [13]:
import pickle

with open(f'../trainData/{NAME}_train_x.pkl', 'wb') as file:
    pickle.dump(train_x, file)
with open(f'../trainData/{NAME}_train_y.pkl', 'wb') as file:
    pickle.dump(train_y, file)
with open(f'../trainData/{NAME}_validation_x.pkl', 'wb') as file:
    pickle.dump(validation_x, file)
with open(f'../trainData/{NAME}_validation_y.pkl', 'wb') as file:
    pickle.dump(validation_y, file)

In [2]:
import pickle

with open(f"../trainData/{NAME}_train_x.pkl", "rb") as file:
    train_x = pickle.load(file)
with open(f"../trainData/{NAME}_train_y.pkl", "rb") as file:
    train_y = pickle.load(file)
with open(f"../trainData/{NAME}_validation_x.pkl", "rb") as file:
    validation_x = pickle.load(file)
with open(f"../trainData/{NAME}_validation_y.pkl", "rb") as file:
    validation_y = pickle.load(file)

# Model:

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [4]:
EPOCHS = 240

batchSize = 96
layers = 2
nodes = 64#256
denseNodes = 32#128

dropOut = 0.4#0.8 #0.92 #0.88
rec_dropout = 0
l1l2_reg = 0 #1e-5#1e-3

learningRate = 0.00001
decay = 0

hp_suffix = f"bs({batchSize})_layers({layers})_noded({nodes})_dNodes({denseNodes})_do({int(dropOut*100)}%)_recdo({rec_dropout})_l1l2({l1l2_reg})_lr({learningRate})_decay({decay})"

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from tqdm.keras import TqdmCallback
import datetime
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import regularizers
import json
# model
model = Sequential()

for _ in range(layers-1):
  model.add(LSTM(nodes, 
               activation="tanh", 
               recurrent_activation = 'sigmoid', 
               recurrent_dropout = rec_dropout, 
               unroll = False, 
               use_bias = True, 
               input_shape=(train_x.shape[1:]), 
               return_sequences=True,
               kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
               #bias_regularizer=regularizers.l2(l1l2_reg),
               activity_regularizer=regularizers.l2(l1l2_reg)
               ))
  model.add(Dropout(dropOut))
  #model.add(BatchNormalization())

model.add(LSTM(nodes, 
             activation="tanh", 
             recurrent_activation = 'sigmoid', 
             recurrent_dropout = rec_dropout, 
             unroll = False, 
             use_bias = True, 
             input_shape=(train_x.shape[1:]),
             kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
             #bias_regularizer=regularizers.l2(l1l2_reg),
             activity_regularizer=regularizers.l2(l1l2_reg)
             ))
model.add(Dropout(dropOut))
#model.add(BatchNormalization())

model.add(Dense(denseNodes, 
                activation="relu",
                kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                #bias_regularizer=regularizers.l2(l1l2_reg), 
                activity_regularizer=regularizers.l2(l1l2_reg)))
model.add(Dropout(dropOut))

model.add(Dense(3, activation="softmax"))



# opt  
opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?


model.compile(loss="sparse_categorical_crossentropy",
              optimizer = opt,
              metrics=["accuracy"])


filename = "" + NAME + "-{epoch:02d}"
filepath = f"models/{filename}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch

# Prepare TensorBoard callback
log_dir = "../logs/fit/" + f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# store hyper-parameters
# Path to the JSON file
file_path = "models/hps/hps.json"
# Load the JSON file into a dictionary
with open(file_path, "r") as json_file:
    data = json.load(json_file)
# Add a new key-value pair to the dictionary
data[f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")] = hp_suffix
# Write the updated dictionary back to the file
with open(file_path, "w") as json_file:
    json.dump(data, json_file, indent=4)




# train
history = model.fit(
  train_x, train_y,
  batch_size = batchSize,
  epochs = EPOCHS,
  validation_data=(validation_x, validation_y),
  callbacks=[checkpoint, TqdmCallback(verbose=0), tensorboard_callback])
  #callbacks=[checkpoint])

c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
  0%|          | 0/240 [00:00<?, ?epoch/s]

Epoch 1/240
461/464 [============================>.] - ETA: 0s - loss: 1.0956 - accuracy: 0.3503

  0%|          | 1/240 [00:09<36:44,  9.23s/epoch, loss=1.1, accuracy=0.35, val_loss=1.09, val_accuracy=0.335]

464/464 [==============================] - 9s 13ms/step - loss: 1.0956 - accuracy: 0.3503 - val_loss: 1.0933 - val_accuracy: 0.3348
Epoch 2/240
462/464 [============================>.] - ETA: 0s - loss: 1.0889 - accuracy: 0.3679

...

 79%|███████▉  | 190/240 [17:01<04:27,  5.35s/epoch, loss=0.999, accuracy=0.483, val_loss=1.02, val_accuracy=0.471]

464/464 [==============================] - 5s 12ms/step - loss: 0.9990 - accuracy: 0.4831 - val_loss: 1.0206 - val_accuracy: 0.4713
Epoch 191/240
463/464 [============================>.] - ETA: 0s - loss: 0.9984 - accuracy: 0.4820

 80%|███████▉  | 191/240 [17:06<04:22,  5.36s/epoch, loss=0.998, accuracy=0.482, val_loss=1.02, val_accuracy=0.468]

464/464 [==============================] - 5s 12ms/step - loss: 0.9984 - accuracy: 0.4821 - val_loss: 1.0219 - val_accuracy: 0.4679
Epoch 192/240
256/464 [===============>..............] - ETA: 2s - loss: 0.9991 - accuracy: 0.4803

KeyboardInterrupt: 